In [ ]:
import fluids as fds
import pandas as pd
import numpy as np
import math
import hx_hydraulic as hxhy
import plotly.graph_objects as go
import scipy.optimize as sco
import matplotlib.pyplot as plt

from CoolProp.CoolProp import PropsSI

In [ ]:
channel = hxhy.duct(shape='rectangular',h=2E-3,w=3E-3,L=1.596)

In [ ]:
inox = hxhy.duct(shape='tubular',D=0.0165,L=5)

In [ ]:
copper = hxhy.duct(shape='tubular',D=0.02,L=13)

In [ ]:
rho = PropsSI('D', 'P', 1, 'T', -15+273.15, f'INCOMP::MPG[0.4]')
eta = PropsSI('V', 'P', 1, 'T', -15+273.15, f'INCOMP::MPG[0.4]')

In [ ]:
rho

In [ ]:
eta*1E6

In [ ]:
bend = hxhy.bend(D=0.016,angle=90, bend_diameters=5, roughness=0.001*1E-3)

In [ ]:
debitmetre = pd.read_csv(r'C:\Users\valen\Downloads\pdc_1_debitmetre.csv')

In [ ]:
panels = pd.read_excel(r'C:\Users\valen\Downloads\V4.413_x10.xlsx')

In [ ]:
popt2, pcov2 = sco.curve_fit(lambda x,a,b : a*x**2+b*x,panels["Vdot"],panels["PL"])

In [ ]:
debitmetre.rename(columns={"x":"Vdot"," y":"PL"},inplace=True)

In [ ]:
popt, pcov = sco.curve_fit(lambda x,a,b : a*x**2+b*x,debitmetre["Vdot"],debitmetre["PL"])

In [ ]:
# create a dataframe with a column Vdot with a list np.linspace(0,5000,10)
df = pd.DataFrame(np.arange(10,200,10)/304,columns=["Vdot"])

In [ ]:
fluid = 'MPG'
glycol_rate = 0.
p = 1.
T = 293.15

rho = PropsSI('D', 'P', p, 'T', T, f'INCOMP::{fluid}[{glycol_rate}]')
eta = PropsSI('V', 'P', p, 'T', T, f'INCOMP::{fluid}[{glycol_rate}]') # kg/m3

In [ ]:
df['channel_PL'] = df['Vdot'].apply(lambda x : channel.regular_PL(x, fluid, glycol_rate, p, T))

In [ ]:
df

In [ ]:
fluent = [13.0,25.1,37.3,49,60.9]

In [ ]:
fluent = fluent + [np.nan] * (len(df) - len(fluent))

In [ ]:
df['fluent'] = fluent

In [ ]:
df

In [ ]:
df['channel_PL'] *= 1000

In [ ]:
df['AE'] = df.apply(lambda x : abs(x['channel_PL'] - x['fluent']),axis=1)
df['RE'] = df.apply(lambda x : abs(x['channel_PL'] - x['fluent'])/x['fluent'],axis=1)

In [ ]:
df

In [ ]:
plt.plot(df['Vdot'],df['channel_PL'],label="channel")
plt.plot(df['Vdot'],df['fluent'],label="fluent")
plt.xlabel("Vdot (L/h)")
plt.ylabel("Pressure loss (Pa)")
plt.xlim(0,0.2)
plt.ylim(0,100)
plt.legend()

In [ ]:
df['channel_PL'] = df['Vdot'].apply(lambda x : channel.regular_PL(x, fluid, glycol_rate, p, T))

In [ ]:
df['inox_PL'] = df['Vdot'].apply(lambda x : inox.regular_PL(x,'MPG',0.4,1,273.15-15))

In [ ]:
df['copper_PL'] = df['Vdot'].apply(lambda x : copper.regular_PL(x,'MPG',0.4,1,273.15-15))

In [ ]:
df['bends_PL'] = df['Vdot'].apply(lambda x : 10*bend.singular_PL(x,'MPG',0.4,1,273.15-15))

In [ ]:
df['debitmetre_PL'] = df['Vdot'].apply(lambda x : popt[0]*x**2+popt[1]*x)

In [ ]:
df['debitmetres_PL'] = df['Vdot'].apply(lambda x : 5*(popt[0]*x**2+popt[1]*x))

In [ ]:
df['V4.413_x10_PL'] = df['Vdot'].apply(lambda x : (popt2[0]*x**2+popt2[1]*x))

In [ ]:
df['min PL'] = df['inox_PL']+df['copper_PL']+df['bends_PL']+df['debitmetres_PL']

In [ ]:
df['max PL'] = df['inox_PL']+df['copper_PL']+df['bends_PL']+df['debitmetres_PL']+df['V4.413_x10_PL']

In [ ]:
# plot all curves in one figure
plt.plot(df['Vdot'],df['inox_PL'],label='inox')
plt.plot(df['Vdot'],df['copper_PL'],label='copper')
plt.plot(df['Vdot'],df['bends_PL'],label='bends')
plt.plot(df['Vdot'],df['debitmetres_PL'],label='debitmetres')
plt.plot(df['Vdot'],df['V4.413_x10_PL'],label='V4.413_x10_PL')
plt.plot(df['Vdot'],df['total_PL'],label='total_PL')
plt.legend()

plt.xlabel('Flow rate (L/h)')
plt.ylabel('Pressure losses (kPa)')

In [ ]:
fig = go.Figure()

# fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['inox_PL'], mode='lines', name='inox'))
fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['copper_PL'], mode='lines', name='copper'))
# fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['bends_PL'], mode='lines', name='bends'))
# fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['debitmetre_PL'], mode='lines', name='debitmetre x1'))
# fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['debitmetres_PL'], mode='lines', name='debitmetres'))
# fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['V4.413_x10_PL'], mode='lines', name='V4.413_x10_PL'))
# fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['total_PL'], mode='lines', name='total_PL'))
fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['min PL'], mode='lines', name='min PL'))
fig = fig.add_trace(go.Scatter(x=df['Vdot'], y=df['max PL'], mode='lines', name='max PL'))

fig = fig.update_layout(title='Pressure losses vs Flow rate',
                  xaxis_title='Flow rate (L/h)',
                  yaxis_title='Pressure losses (kPa)')

fig = fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        l=0,
        r=0,
        b=50,
        t=50,
        pad=1
    ),
)

# limiter x à à 3000
fig = fig.update_xaxes(range=[0, 3000])


fig.show()


In [ ]:
fig.write_html('BTP theoretical max pressure losses.html')

In [ ]:
list_Q_L = np.array([64,152,245,319,402,449,597])
list_Q = list_Q_L/3600000 # m3/s

In [ ]:
list_PL = []
list_PL_bend = []

In [ ]:
Dx = 0.0165
Ax = math.pi*(Dx/2)**2
V = (700/(3.6*1E6))/Ax
L= 540/100

Re = fds.core.Reynolds(V,Dx,1024,mu=0.002735) # viscosité dynamique mu ou eta)
f = fds.friction.friction_factor(Re = Re,eD=0.001/Dx)
K = fds.K_from_f(f,L,Dx)
dP = fds.dP_from_K(K,rho=1024,V=V)
print(dP/1000)

In [ ]:
Dx = 0.025
Ax = math.pi*(Dx/2)**2
V = (1000/(3.6*1E6))/Ax
L= 870/100

Re = fds.core.Reynolds(V,Dx,1024,mu=0.002735) # viscosité dynamique mu ou eta)
f = fds.friction.friction_factor(Re = Re,eD=0.001/Dx)
K = fds.K_from_f(f,L,Dx)
dP = fds.dP_from_K(K,rho=1024,V=V)
print(dP/1000)

In [ ]:
dP_bend = fds.dP_from_K(fds.fittings.bend_rounded_Crane(Dx,angle=90.,bend_diameters=2.),rho=1000,V=V)

In [ ]:
for i in range(len(list_Q)):
    u = list_Q[i]/A
    Re = fds.core.Reynolds(u,Dx,1000,mu=1*1E-3) # viscosité dynamique mu ou eta
    f = fds.friction.friction_factor(Re = Re,eD=0.001/D)
    K = f*(L/D)
    list_PL.append((1000/2)*K*u**2)

    K2 = fds.fittings.bend_rounded_Crane(D,angle=90.,bend_diameters=2.)

    list_PL_bend.append((1000/2)*K2*u**2)


In [ ]:
list_PL_kPa = np.array(list_PL)/1000

In [ ]:
list_PL_kPa

In [ ]:
list_PL

In [ ]:
list_PL_bend